# TF-IDF 실습

In [160]:
from sklearn.feature_extraction.text import TfidfVectorizer


doc_list = [
    '프리시즌 아시아 투어를 떠나는 토트넘은 싱가포르, 중국을 차례로 방문해 ICC 경기를 치른다.',
    '영국 "풋볼 런던"은 11일 "토트넘이 ICC 첫 경기에서 가장 강력한 스쿼드로 유벤투스에 맞설 것"이라고 평가했다.',
    '토트넘에 합류하는 손흥민은 유벤투스전 출전을 목표로 구슬땀을 흘릴 예정이다.',
]

tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_list)
doc_similarities = (tfidf_matrix * tfidf_matrix.T)

print(doc_similarities.toarray())

[[1.         0.04612956 0.        ]
 [0.04612956 1.         0.        ]
 [0.         0.         1.        ]]


In [161]:
from konlpy.tag import Okt


okt = Okt()

doc_nouns_list = [' '.join(okt.nouns(doc)) for doc in doc_list]
print(doc_nouns_list)

tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)
doc_nouns_similarities = (tfidf_matrix * tfidf_matrix.T)

print(doc_nouns_similarities.toarray())

['프리 시즌 아시아 투어 토트넘 싱가포르 중국 차례 방문 경기', '영국 풋볼 런던 은 토트넘 첫 경기 가장 스 쿼드로 유벤투스 것 평가', '토트넘 합류 손흥민 유벤투스 전 출전 목표 구슬 땀 예정']
[[1.         0.11327504 0.04435806]
 [0.11327504 1.         0.12859167]
 [0.04435806 0.12859167 1.        ]]


# KoNLP 활용 리뷰 분류

## 필요한 정보 불러오기

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Kkma
from konlpy.tag import Okt
from konlpy.utils import pprint
import json
import pickle

In [2]:
con = sqlite3.connect("db.sqlite3")
cur = con.cursor()
# read table 
df_store = pd.read_sql_query("SELECT * from api_store", con)
df_review = pd.read_sql_query("SELECT * from api_storereview", con)

# clonse DB connection
con.close()

In [3]:
df_review.head()

,id,total_score,content,reg_time,store_id,user_id
0,1,5.0,전포 윗길에 새로 생긴! 호주에서 온 쉐프가 직접 요리하는 호주식 레스토랑!,2020-04-28 09:17:35.263075,15,68632
1,2,5.0,샌드위치 내용물도 알차게 들어있고 맛있었어요 가성비 추천,2020-04-28 09:17:35.263075,18,389728
2,3,4.0,홈플러스 1층 매장 푸드코트 내 있는 매장인데 계란찜정식은 처음보고 시켜봣는데 사진...,2020-04-28 09:17:35.263075,19,68716
3,4,2.0,"전 여기 5년전에 가봤었는데 그때 기억은 별로였어요\n단체손님이 많았고, 차려지는건...",2020-04-28 09:17:35.263075,37,774353
4,5,3.0,친구들끼리 술 간단하게마시러 감. 스끼다시 괜찮지만 회 양이 조금 부족한 느낌. 맛...,2020-04-28 09:17:35.263075,38,115682


In [4]:
df_store.head()

,id,store_name,branch,area,tel,address,latitude,longitude,category,review_count,review_total_score,addr
0,1,Agal,nan,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,아구찜|포장마차,0,0,서울특별시
1,2,Assisy,nan,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,카페,0,0,광주광역시
2,3,Battered Sole,nan,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,피쉬앤칩스|펍,0,0,서울특별시
3,4,Chakyoung,nan,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,레스토랑|카프레제,0,0,부산광역시
4,5,Delabobo,nan,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,디저트카페|디저트,0,0,서울특별시


In [24]:
df_store = df_store[df_store["addr"]=="충청북도 충주시"]

In [22]:
cities = set(df_store["addr"])

In [23]:
cities

{'강원도 강릉시',
 '강원도 고성군',
 '강원도 동해시',
 '강원도 삼척시',
 '강원도 속초시',
 '강원도 양구군',
 '강원도 양양군',
 '강원도 영월군',
 '강원도 원주시',
 '강원도 인제군',
 '강원도 정선군',
 '강원도 철원군',
 '강원도 춘천시',
 '강원도 태백시',
 '강원도 평창군',
 '강원도 홍천군',
 '강원도 화천군',
 '강원도 횡성군',
 '경기도 가평군',
 '경기도 고양시',
 '경기도 과천시',
 '경기도 광명시',
 '경기도 구리시',
 '경기도 군포시',
 '경기도 김포시',
 '경기도 남양주시',
 '경기도 동두천시',
 '경기도 부천시',
 '경기도 성남시',
 '경기도 수원시',
 '경기도 시흥시',
 '경기도 안산시',
 '경기도 안성시',
 '경기도 안양시',
 '경기도 양주시',
 '경기도 양평군',
 '경기도 여주시',
 '경기도 연천군',
 '경기도 오산시',
 '경기도 용인시',
 '경기도 의왕시',
 '경기도 의정부시',
 '경기도 이천시',
 '경기도 파주시',
 '경기도 평택시',
 '경기도 포천시',
 '경기도 하남시',
 '경기도 화성시',
 '경상남도 거제시',
 '경상남도 거창군',
 '경상남도 고성군',
 '경상남도 김해시',
 '경상남도 남해군',
 '경상남도 밀양시',
 '경상남도 사천시',
 '경상남도 산청군',
 '경상남도 양산시',
 '경상남도 의령군',
 '경상남도 진주시',
 '경상남도 창녕군',
 '경상남도 창원시',
 '경상남도 통영시',
 '경상남도 하동군',
 '경상남도 함안군',
 '경상남도 함양군',
 '경상남도 합천군',
 '경상북도 경산시',
 '경상북도 경주시',
 '경상북도 고령군',
 '경상북도 구미시',
 '경상북도 군위군',
 '경상북도 김천시',
 '경상북도 문경시',
 '경상북도 봉화군',
 '경상북도 상주시',
 '경상북도 성주군',
 '경상북도 안동시',
 '경상북도 영덕군',
 '경상북도 영양군',
 '경상북도 영주시',
 '경상북

In [26]:
# select some columns
stores = df_store[['id','store_name', 'area', 'category']]
reviews = df_review[['user_id','store_id','total_score', 'content']]

# rename columns and return data
reviews.rename(columns={'total_score':'rating'}, inplace=True)
stores.rename(columns={'id':'store_id'},inplace=True)

In [27]:
reviews.head()

,user_id,store_id,rating,content
0,68632,15,5.0,전포 윗길에 새로 생긴! 호주에서 온 쉐프가 직접 요리하는 호주식 레스토랑!
1,389728,18,5.0,샌드위치 내용물도 알차게 들어있고 맛있었어요 가성비 추천
2,68716,19,4.0,홈플러스 1층 매장 푸드코트 내 있는 매장인데 계란찜정식은 처음보고 시켜봣는데 사진...
3,774353,37,2.0,"전 여기 5년전에 가봤었는데 그때 기억은 별로였어요\n단체손님이 많았고, 차려지는건..."
4,115682,38,3.0,친구들끼리 술 간단하게마시러 감. 스끼다시 괜찮지만 회 양이 조금 부족한 느낌. 맛...


In [28]:
stores.head()

,store_id,store_name,area,category
427,428,13스시,충주,차돌박이|초밥
454,455,153낙지유,충주,연포탕|산낙지
892,893,1미터피자,충주,nan
1133,1134,21세기,교현동,nan
1485,1486,24시전주명가콩나물국밥,충주,콩나물국밥|전주콩나물국밥


In [29]:
stores[["store_id", "store_name"]]

,store_id,store_name
427,428,13스시
454,455,153낙지유
892,893,1미터피자
1133,1134,21세기
1485,1486,24시전주명가콩나물국밥
...,...,...
459107,459212,해피돈스
459191,459296,해피오네치킨
459192,459297,해피오네치킨
459194,459299,해피오네치킨


In [30]:
reviews = pd.merge(stores, reviews, how='outer')

In [31]:
reviews.head()

,store_id,store_name,area,category,user_id,rating,content
0,428,13스시,충주,차돌박이|초밥,NaN,NaN,NaN
1,455,153낙지유,충주,연포탕|산낙지,NaN,NaN,NaN
2,893,1미터피자,충주,nan,NaN,NaN,NaN
3,1134,21세기,교현동,nan,NaN,NaN,NaN
4,1486,24시전주명가콩나물국밥,충주,콩나물국밥|전주콩나물국밥,NaN,NaN,NaN


## 벡터화

In [33]:
store_content = {}
kkma = Kkma()
okt = Okt()

In [61]:
store_content = {}
for idx, review in reviews.iterrows():
    if review['content']:
        content = [noun for noun in (kkma.nouns(f"u{review['content']}"))]
        categories = kkma.nouns(f"u{review['category']}")
        for c in categories:
            content.append(c)
        content.append(review['area'])
        if store_content[review['store_id']]:
            store_content[review['store_id']] = content
        else:
            store_content[review['store_id']] = content

In [108]:
store_content = {}
for idx, review in reviews.iterrows():
    if review['content']:
        content = [noun for noun in (kkma.nouns(f"u{review['content']}"))]
        content.extend(kkma.nouns(f"u{review['category']}"))
        content.append(review['area'])
        store_content[review['store_id']] = content

In [34]:
new_store_content = {}
for idx, review in reviews.iterrows():
    if review['content']:
        content = [noun for noun in (kkma.nouns(f"u{review['content']}"))]
        content.extend(kkma.nouns(f"u{review['category']}"))
        content.append(review['area'])
        if new_store_content.get(review['store_id']):
            new_store_content[review['store_id']].extend(content)
        else:
            new_store_content[review['store_id']] = content

KeyboardInterrupt: 

In [154]:
print(len(new_store_content))

456599


In [ ]:
for key, item in store_content.items():
    print(f'{key} : {item}')

In [158]:
with open('kdump.json', 'w', encoding='utf-8') as make_file:
    json.dump(new_store_content, make_file, indent="\t")

In [159]:
with open('kdump.json', 'r') as f:
    json_data = json.load(f)

In [ ]:
i = 0
for key, item in json_data.items():
    print(f'{key} : {item}')

In [210]:
json_data

{'1': ['아구', '아구찜', '찜', '포장마차', '홍대'],
 '2': ['카페', '광주'],
 '3': ['피', '피쉬앤칩스', '쉬', '앤', '칩스', '펍', '이태원'],
 '4': ['레스토랑', '카프', '카프레', '레', '달맞이길'],
 '5': ['디저트', '디저트카페', '카페', '발산역'],
 '6': ['펍', '하남'],
 '7': ['마들렌', '베이커리', '만촌동'],
 '8': ['남포동'],
 '9': ['고', '고깃집', '깃', '집', '고기', '고기집', '진주'],
 '10': ['중식', '레스토랑', '신촌'],
 '11': ['국수', '고기', '고기국수', '월곶'],
 '12': ['역삼역'],
 '13': ['수영역'],
 '14': ['오뎅', '오뎅탕', '탕', '참치', '유성'],
 '15': ['전포',
  '윗길',
  '호주',
  '쉐프',
  '요리',
  '호',
  '호주식',
  '주식',
  '레스토랑',
  '호주',
  '호주레스토랑',
  '레스토랑',
  '전포동'],
 '16': ['에스프레소', '사당'],
 '17': ['건대'],
 '18': ['샌드위치', '내용물', '성비', '추천', '샌드위치', '쥬스', '전주'],
 '19': ['홈',
  '홈플러스',
  '플러스',
  '1',
  '1층',
  '층',
  '매장',
  '푸드',
  '푸드코트',
  '코트',
  '내',
  '계란',
  '계란찜정식',
  '찜',
  '정식',
  '처음',
  '봣',
  '사진',
  '치즈',
  '계란찜',
  '메인',
  '위',
  '맛',
  '재미',
  '성비',
  '차',
  '곳',
  '전주',
  '전주비빔밥',
  '비빔밥',
  '향남지구'],
 '20': ['서천동'],
 '21': ['까페', '플라워', '플라워카페', '카페', '신건지동'],
 '22': ['방배동'],
 '23': ['내손

In [195]:
tfidf = TfidfVectorizer(max_features = 100, max_df=0.95, min_df=0)

#generate tf-idf term-document matrix
A_tfidf_sp = tfidf.fit_transform(documents)  #size D x V

In [196]:
#tf-idf dictionary    
tfidf_dict = tfidf.get_feature_names()
print(tfidf_dict)

['nan', '가게', '가격', '갈비', '고기', '고기집', '곱창', '구이', '국물', '국밥', '국수', '김밥', '김치', '꼬치', '냉면', '느낌', '닭갈비', '대구', '돈까스', '돼지', '돼지갈비', '디저트', '떡볶이', '레스토랑', '리필', '만두', '맛집', '맥주', '메뉴', '무한', '반찬', '방문', '버거', '보쌈', '볶음', '볶음밥', '분위기', '불고기', '브런', '비빔밥', '사람', '사장님', '삼겹살', '샐러드', '생각', '서비스', '성비', '소고기', '손님', '수제', '순대', '술집', '스테이크', '시간', '식당', '식사', '양념', '여기', '요리', '유명', '음식', '이자', '일산', '있음', '장어', '전골', '전문', '전주', '정도', '정식', '족발', '주문', '주차', '중국집', '직원', '짬뽕', '찌개', '처음', '청주', '초밥', '추천', '치즈', '치킨', '친절', '카페', '칼국수', '커피', '케이크', '크림', '탕수육', '테이블', '튀김', '파스타', '피자', '하나', '한우', '해물', '해장국', '홍대', '횟집']


In [197]:
data_array = A_tfidf_sp.toarray()
data = pd.DataFrame(data_array, columns=tfidf_dict)
data.shape

(456599, 100)

In [ ]:
cosine_similarities = linear_kernel(A_tfidf_sp, A_tfidf_sp)

In [228]:
json_list = []
for key, item in json_data.items():
    new_item = ''
    for s in item:
        new_item += ' ' + s
    json_list.append([key, new_item])


In [229]:
data = pd.DataFrame(json_list, columns=['store_id', 'content'])

In [231]:
tf = TfidfVectorizer()
tfidf_matrix = tf.fit_transform(data['content'])

In [240]:
print(data['content'])

0              아구 아구찜 찜 포장마차 홍대
1                         카페 광주
2          피 피쉬앤칩스 쉬 앤 칩스 펍 이태원
3            레스토랑 카프 카프레 레 달맞이길
4              디저트 디저트카페 카페 발산역
                  ...          
456594                      녹양동
456595                      안중읍
456596                      퇴촌면
456597                      소주동
456598                     의정부동
Name: content, Length: 456599, dtype: object


In [ ]:
tfidfv = TfidfVectorizer().fit(data['content'])
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

In [ ]:
print(tfidfv.transform(corpus))

In [235]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
results = {}
for idx, row in ds.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
   similar_items = [(cosine_similarities[idx][i], data['store_id'][i]) for i in similar_indices] 
   results[row['id']] = similar_items[1:]


MemoryError: Unable to allocate 26.3 GiB for an array with shape (3536218231,) and data type int64

# OKT

In [4]:
okt = Okt()

In [5]:
okt_content = {}
for idx, review in reviews.iterrows():
    if review['content']:
        content = [noun for noun in (okt.nouns(f"u{review['content']}"))]
        content.extend(okt.nouns(f"u{review['category']}"))
        content.append(review['area'])
        if okt_content.get(review['store_id']):
            okt_content[review['store_id']].extend(content)
        else:
            okt_content[review['store_id']] = content

NameError: name 'reviews' is not defined

In [254]:
with open('okt_data.json', 'w', encoding='utf-8') as make_file:
    json.dump(okt_content, make_file, indent="\t")

In [14]:
from sklearn.metrics.pairwise import linear_kernel

In [8]:
with open('okt_data.json', 'r') as f:
    okt_data = json.load(f)

In [9]:
okt_list = []
for key, item in okt_data.items():
    new_item = ''
    for s in item:
        new_item += ' ' + s
    okt_list.append([key, new_item])


In [10]:
data = pd.DataFrame(okt_list, columns=['store_id', 'content'])

In [11]:
tf = TfidfVectorizer(min_df=50)
tfidf_matrix = tf.fit_transform(data['content'])

In [12]:
tfidf_matrix.shape

(456599, 3951)

In [15]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
results = {}
for idx, row in ds.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
   similar_items = [(cosine_similarities[idx][i], data['store_id'][i]) for i in similar_indices] 
   results[row['id']] = similar_items[1:]


MemoryError: Unable to allocate 24.8 GiB for an array with shape (3327147044,) and data type float64

In [1]:
corpus = data['content']
tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

NameError: name 'data' is not defined